In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import requests
from selenium.webdriver.chrome.options import Options


BASE_URL = "https://www.leonardoshoes.com"
CATEGORIES = ["/collections/donna-calzature-mocassini",
              "/collections/donna-calzature-decolte-tacchi",
              "/collections/donna-calzature-sandali",
              "/collections/donna-calzature-mules",
              "/collections/donna-calzature-ballerine",
              "/collections/donna-calzature-stivali-tronchetti",
              "/collections/donna-calzature-sneakers", 
              "/collections/donna-calzature-sandali",
              "/collections/donna-calzature-sandali?page=2"
              "/collections/donna-sandali-tacco-alto",
              "/collections/donna-sandali-tacco-basso",
              "/collections/slingback",
              "/collections/mocassini-da-barca",
              "/collections/polacchine-donna-in-pelle-artigianali"
              ]

################################################# N O T E S #####################################################
#                    Requests library doesn't work on this website, use webdriver instead                       #
#             Pages with more than 100 products need to be scraped with suffix ?page={number_of_page}           #
#                  There are product names containing "," which can interfere with csv export                   #
#################################################################################################################

In [ ]:
df_leonardo = pd.DataFrame(columns = ["nome", "categoria", "prezzo", "tacco", "taglie", "colore", "link"])

# Cicle through category pages
for CAT in CATEGORIES:
    url = BASE_URL + CAT
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options)
    driver.get(url)
    page = driver.page_source
    driver.quit()

    bs = BeautifulSoup(page, 'html.parser')
    links = bs.find_all("div", class_ = "h4 spf-product-card__title")
    l_shoe_links = []

    # Cycle for taking the link, put into links
    for link in links:
        l_shoe_links.append(BASE_URL + link.a["href"])
    
    # Cycle through every shoe's page
    for shoe_link in l_shoe_links:
        print(shoe_link)
        response = requests.get(shoe_link)
        bs_shoe = BeautifulSoup(response.content, 'html.parser') 
        product_name = bs_shoe.find("h1").text.replace("," , "") # Ci sono prodotti con una virgola, danno fastidio al csv
        price = bs_shoe.find("span", class_ = "price price--large").text.split("vendita")[1] if bs_shoe.find("span", class_ = "price price--large") else bs_shoe.find("span", class_ = "price price--highlight price--large").text.split("vendita")[1]
        temp = bs_shoe.find_all("label", class_ = "block-swatch__item")
        sizes = []

        for taglia in temp:
            sizes.append(taglia.text)
        sizes = " ".join(sizes)
        bs_details = bs_shoe.find_all("div", class_ = "product-tabs__tab-item-content rte")[1]
        details = bs_details.find_all("p")
        heel = details[6].text.replace(" cm", "").split(": ")[1] if len(details) > 6 else "NaN"
        color = details[0].text.split(": ")[1]
        
        row = pd.Series({
                        "nome": product_name, 
                        "categoria": CAT,
                        "prezzo": price, 
                        "tacco": heel, 
                        "taglie": sizes, 
                        "colore": color,
                        "link": shoe_link})
        df = pd.DataFrame(row).T
            
        # Inserisco nuova riga
        df_leonardo = pd.concat([df_leonardo, df])

In [ ]:
df_leonardo["categoria"].unique()

array(['/collections/donna-calzature-mocassini',
       '/collections/donna-calzature-sandali',
       '/collections/donna-calzature-mules',
       '/collections/donna-calzature-ballerine',
       '/collections/donna-calzature-stivali-tronchetti',
       '/collections/donna-calzature-stivali',
       '/collections/donna-calzature-stringate',
       '/collections/donna-calzature-sandali?page=2/collections/donna-sandali-tacco-alto',
       '/collections/donna-sandali-tacco-basso', '/collections/slingback',
       '/collections/donna-stivali-texani',
       '/collections/polacchine-donna-in-pelle-artigianali'], dtype=object)

In [ ]:
df_leonardo.to_csv("leonardo.csv")

In [ ]:
df_leonardo.loc[df_leonardo["categoria"].str.contains("page")].count()

nome         18
categoria    18
prezzo       18
tacco        18
taglie       18
colore       18
link         18
dtype: int64

,nome,categoria,prezzo,tacco,taglie,colore,link
0,Stivaletto anfibio da donna con dettagli brogu...,/collections/polacchine-donna-in-pelle-artigia...,"€249,00",3,35 36 37 38 39 40 41,Nero,https://www.leonardoshoes.com/collections/pola...
